In [ ]:
pip install pymongo

In [ ]:
# Required libraries
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from pymongo import MongoClient

In [303]:
# Connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador']

In [304]:
# Base URL of the site
SITE_URL = 'https://www.elespectador.com'

In [305]:
DRIVER_PATH = './geckodriver'

In [306]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

C:\Users\Jonat\AppData\Local\Temp/ipykernel_10272/1910320475.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [307]:
browser.get(SITE_URL)

In [308]:
def make_request(browser, relative_path):
    browser.get(SITE_URL + relative_path)
    
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;') 
        if height == check_height:
            break
        check_height = height
    
    return BeautifulSoup(browser.page_source, 'html.parser')

In [309]:
# Converting string date to date ISO
from datetime import datetime

def convertDateToMongo(string_date):
    aux_date = string_date.split(' ') 
    year = aux_date[2]
    month = aux_date[1][:-1]
    day = aux_date[0]
    
    if(month == 'sept'):
        month = 'sep'
    
    hour = aux_date[5].split(':')[0]
    minute =  aux_date[5].split(':')[1]
    aux_horary = aux_date[6].split('.')
    aux2_horary = aux_horary[0]+aux_horary[1].split(' ')[0]
    horary = aux2_horary[0].upper()+aux2_horary[2].upper()
    
    final_date = day+' '+month+' '+year+' '+hour+' '+minute+''.join(map(str, horary))
    
    try:
        datetime_object = datetime.strptime(final_date, '%d %b %Y %I %M%p')
    except:
        datetime_object = 'Not available'
        
    print(datetime_object)
    
    return datetime_object
    

In [310]:
def obtainHTML(browser, url_dir):
    current_page = ''
    for x in range(5):
        # Getting HTML content for section news page
        soup = make_request(browser, url_dir+str(current_page))

        # Finding section where news are located
        layout = soup.find(class_ = 'Layout-flexAds')

        # Getting blocks from layout
        blocks = layout.find('section').find_all(recursive = False)
        print(len(blocks)) # 3 blocks founded, 2 for news and 1 for pagination
        cards = blocks[0].find_all(class_ = 'Card' ) + blocks[1].find_all(class_ = 'Card')
        len(cards)

        # Building a list with title and relative path of each news founded
        news = []
        for card in cards:
            news.append({
                'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
                'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
            })
        
        print(news)
        
        for n in news:
            # Getting HTML content for news page
            soup = make_request(browser, n['relative_path'])

            # Extracting news metadata
            n['datetime'] = convertDateToMongo(soup.find(class_ = 'ArticleHeader-Date').get_text())

            try:
                n['author'] = soup.find(class_ = 'ACredit-Author').find('a').get_text()
            except:
                n['author'] = 'Not available'

            try:
                n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
            except: 
                n['summary'] = 'Not available'

            # Extracting and concatenating news full text
            paragraphs = soup.find_all(class_ = 'font--secondary')
            n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

            # Deleting WebStorage
            browser.execute_script("sessionStorage.clear();")
            browser.execute_script("localStorage.clear();")
            
        # print(news)
        collection.insert_many(news)
        
        browser.get(SITE_URL+url_dir+str(current_page))
        time.sleep(3)

        # Get current page in order to advance to the next page
        pag_button = browser.find_element_by_class_name('Pagination-Input')
        current_page = int(pag_button.get_attribute('value'))+1
        print('pagina siguiente: '+current_pag)
        browser.get(SITE_URL+url_dir+str(current_page))
        time.sleep(3)
    

In [311]:
list_urls = ['/archivo/educacion/',
             '/archivo/salud/',
            '/archivo/judicial/'
            ,'/archivo/ciencia/'
            ,'/archivo/investigacion/']

In [312]:
for u in list_urls:
    obtainHTML(browser, u)
    

3
[{'title': 'La Asamblea de Antioquia otorga reconocimiento a la IU Digital', 'relative_path': '/contenido-patrocinado/la-asamblea-de-antioquia-otorga-reconocimiento-a-la-iu-digital/'}, {'title': '“Tenemos que evitar hablar de una generación perdida en educación”', 'relative_path': '/educacion/tenemos-que-evitar-hablar-de-una-generacion-perdida-en-educacion/'}, {'title': 'Comenzaron las citaciones para la última prueba Saber Pro del año ', 'relative_path': '/educacion/comenzaron-las-citaciones-para-la-ultima-prueba-saber-pro-del-ano/'}, {'title': '22.000 niños dejaron de recibir educación preescolar en pandemia', 'relative_path': '/educacion/22000-ninos-dejaron-de-recibir-educacion-preescolar-en-pandemia/'}, {'title': 'Los niños y niñas quieren volver a clases presenciales', 'relative_path': '/educacion/los-ninos-y-ninas-quieren-volver-a-clases-presenciales-segun-sondeo/'}, {'title': 'Aprender jugando para llegar al Sol', 'relative_path': '/contenido-patrocinado/aprender-jugando-para-

C:\Users\Jonat\AppData\Local\Temp/ipykernel_10272/3231225779.py:58: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  pag_button = browser.find_element_by_class_name('Pagination-Input')


pagina siguiente: 3
2
[{'title': 'Invisibles: la infancia en Colombia durante la pandemia', 'relative_path': '/educacion/invisibles-la-infancia-en-colombia-durante-la-pandemia-documental/'}, {'title': 'La IU Digital de Antioquia continúa entregando profesionales comprometidos ', 'relative_path': '/contenido-patrocinado/la-iu-digital-de-antioquia-continua-entregando-profesionales-comprometidos/'}, {'title': '“Niños y niñas pagaron un precio muy alto con el coronavirus”: Germán Casas', 'relative_path': '/educacion/coronavirus-ninos-y-ninas-pagaron-un-precio-muy-alto/'}, {'title': 'La Universidad ECCI impulsa la formación de líderes en administración ', 'relative_path': '/contenido-patrocinado/la-universidad-ecci-impulsa-la-formacion-de-lideres-en-administracion/'}, {'title': 'Fecode se suma a jornada de paro nacional convocada para el 28 de septiembre', 'relative_path': '/educacion/fecode-se-suma-a-jornada-de-paro-nacional-convocada-para-el-28-de-septiembre/'}, {'title': 'Gerencia de mer

WebDriverException: Message: Reached error page: about:neterror?e=dnsNotFound&u=https%3A//www.elespectador.comhttps//reportajes.elespectador.com/arc/derechos-de-la-naturaleza/atrato-2.html&c=UTF-8&d=No%20podemos%20conectar%20al%20servidor%20en%20www.elespectador.comhttps.
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:181:5
UnknownError@chrome://remote/content/shared/webdriver/Errors.jsm:488:5
checkReadyState@chrome://remote/content/marionette/navigate.js:64:24
onNavigation@chrome://remote/content/marionette/navigate.js:312:39
emit@resource://gre/modules/EventEmitter.jsm:160:20
receiveMessage@chrome://remote/content/marionette/actors/MarionetteEventsParent.jsm:42:25
